In [1]:
import jovian
import requests
from bs4 import BeautifulSoup
import pandas as pd
from PIL import Image
from pytesseract import pytesseract
from collections import defaultdict

<IPython.core.display.Javascript object>

In [2]:
keyList = ["Name",
            "Born",
            "Died",
            "Hometown",
            "Biography",
            "Image",
            "Genre",
            "Website",
            "Influences",  
            "Rating",
            "Rating_count",
            "Review_count",
            "Works",          
            "Typical_series"
]

In [6]:
authors=[]
file = open('authors.txt', "r")
lines = file.readlines()
lines = list(set(lines))

count = 0
for line in lines:
    author = {key: None for key in keyList}
    count += 1
    print("Line{}: {}".format(count, line.strip()))
    # download the web page using requests
    author_info = line.strip()
    header = {'User_Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'}
    response = requests.get(author_info,headers=header)

    # capture the text content of the page from response
    page_contents = response.text

    # Save the page as html to avoid scrapping multiple times while experimenting
    with open('author.html', 'w', encoding="utf-8") as file:
        file.write(page_contents)

    # read it html page to scrape information
    with open('author.html', 'r', encoding="utf-8") as file:
        html_source = file.read()

    # The BeautifulSoup returns a document which we shall use to extract information
    doc_author = BeautifulSoup(html_source, 'html.parser')

    for br in doc_author.select("br"):
        br.replace_with("\n")

    authorName = doc_author.find('h1', class_='authorName')
    author['Name'] = authorName.text.strip()

    author_hometown = doc_author.find('div', class_='dataTitle').next_sibling
    author['Hometown'] = author_hometown.text.strip()

    data = doc_author.find_all('div', class_='dataTitle')
    for i in data:
        info = i.text.strip()
    
        if info == 'Website':
            i = i.find_next_sibling('div')     
            author[info] = i.find('a').get('href')
        elif info == 'Influences':
            i = i.find_next_sibling('div')
            a = i.find('span', attrs={'style':'display:none'}) 
            if a != None:
                author[info] = a.text.strip()
            else:
                author[info] = i.text.strip()
        else:
            i = i.find_next_sibling('div')
            attr = i['class']     
            if attr == ['dataItem']:    
                author[info] = i.text.strip()
    
    author_img = doc_author.find('div', class_='leftContainer authorLeftContainer')
    author['Image'] = author_img.find('img').get('src')

    author_info = doc_author.find('div', class_='aboutAuthorInfo').find('span')
    author['Biography'] = author_info.text.strip()

    author_data = doc_author.find('div', class_='hreview-aggregate')
    for i in author_data.find_all('span'):
        check = i.get('itemprop')
        if check == 'ratingValue':
            author['Rating'] = i.text.strip()
        elif check == 'ratingCount':
            author['Rating_count'] = i.text.strip()
        elif check == 'reviewCount':
            author['Review_count'] = i.text.strip()
            author_work = i.find_next('a')
            author_book = author_work.get('href')
            author['Works'] = author_work.text.strip().split()[0]

    author['Typical_series'] = []
    typicalSeries = doc_author.find_all('div', class_='seriesDesc')
    for series in typicalSeries:
        series = series.find('a', class_='bookTitle')
        series = series.text.strip()
        author['Typical_series'].extend([series])

    print(author)
    authors.append(author)


Line1: https://www.goodreads.com/author/show/13661.Victor_Hugo
{'Name': 'Victor Hugo', 'Born': 'February 26, 1802', 'Died': 'May 22, 1885', 'Hometown': 'in Besançon, Doubs, France', 'Biography': 'After Napoleon III seized power in 1851, French writer Victor Marie Hugo went into exile and in 1870 returned to France; his novels include \nThe Hunchback of Notre Dame\n (1831) and \nLes Misérables\n (1862). \n\nThis poet, playwright, novelist, dramatist, essayist, visual artist, statesman, and perhaps the most influential, important exponent of the Romantic movement in France, campaigned for human rights. People in France regard him as one of greatest poets of that country and know him better abroad.', 'Image': 'https://images.gr-assets.com/authors/1415946858p5/13661.jpg', 'Genre': 'Literature & Fiction, Poetry, Philosophy', 'Website': None, 'Influences': 'William Shakespeare, Chateaubriand, Walter Scott, Lamartine, Dante', 'Rating': '4.15', 'Rating_count': '1,093,968', 'Review_count': '37,

In [7]:
authors_df = pd.DataFrame(data=authors)
authors_df

,Name,Born,Died,Hometown,Biography,Image,Genre,Website,Influences,Rating,Rating_count,Review_count,Works,Typical_series,Member Since,Twitter
0,Victor Hugo,"February 26, 1802","May 22, 1885","in Besançon, Doubs, France","After Napoleon III seized power in 1851, Frenc...",https://images.gr-assets.com/authors/141594685...,"Literature & Fiction, Poetry, Philosophy",None,"William Shakespeare, Chateaubriand, Walter Sco...",4.15,"1,093,968","37,496","6,584","[Les Misérables, Los Miserables, Notre Dame de...",NaN,NaN
1,Shel Silverstein,"September 25, 1930","May 10, 1999","in Chicago, IL, The United States",Shel Silverstein was the author-artist of many...,https://images.gr-assets.com/authors/169071082...,"Poetry, Children's Books",http://www.shelsilverstein.com/indexSite.html,None,4.35,"3,121,533","44,393",141,"[The Missing Piece, Runny Babbit]",NaN,NaN
2,Douglas Adams,"March 11, 1952","May 11, 2001","in Cambridge, England, The United Kingdom","Douglas Noël Adams was an English author, comi...",https://images.gr-assets.com/authors/161627770...,"Science Fiction, Mystery, Humor",http://www.douglasadams.com/,"Monty Python, Kurt Vonnegut, P. G. Wodehouse, ...",4.2,"3,311,148","82,136",170,"[The Hitchhiker's Guide to the Galaxy, Dirk Ge...",NaN,NaN
3,J.D. Salinger,"January 01, 1919","January 27, 2010","in Manhattan, New York, The United States","Works, most notably novel \nThe Catcher in the...",https://images.gr-assets.com/authors/128877767...,"Literature & Fiction, Short Stories",None,"Sherwood Anderson, Anton Chekhov, F. Scott Fit...",3.83,"3,851,401","100,709",146,[Babe Gladwaller],NaN,NaN
4,Cassandra Clare,None,None,"Tehran, Iran",Cassandra Clare was born to American parents i...,https://images.gr-assets.com/authors/168934731...,Science Fiction & Fantasy,http://www.cassandraclare.com/,None,4.22,"8,855,082","517,724",187,"[The Mortal Instruments, The Infernal Devices,...",December 2008,NaN
5,Ray Bradbury,"August 22, 1920","June 05, 2012","in Waukegan, Illinois, The United States","Ray Douglas Bradbury, American novelist, short...",https://images.gr-assets.com/authors/144595595...,"Literature & Fiction, Science Fiction & Fantas...",http://www.raybradbury.com/,"Edgar Rice Burroughs, John Steinbeck, Jules Ve...",3.99,"3,176,439","131,297","2,245","[Green Town, Crumley Mysteries, Bradbury Stori...",NaN,NaN
6,Stephen Chbosky,"January 25, 1970",None,"in Pittsburgh, The United States","Stephen Chbosky grew up in Pittsburgh, Pennsyl...",https://images.gr-assets.com/authors/159913296...,"Fiction, Young Adult, Horror",None,None,4.21,"1,775,584","96,839",16,"[Imaginary Friend, Imaginary Friend]",NaN,NaN
7,E.B. White,"July 11, 1899","October 01, 1985","in Mount Vernon, NY, The United States",Elwyn Brooks White was a leading American essa...,https://images.gr-assets.com/authors/119851941...,"Children's Books, Literature & Fiction, Nonfic...",None,None,4.18,"2,088,872","36,343",108,[],NaN,NaN
8,Kathryn Stockett,None,None,"Jackson, Mississippi, The United States",Kathryn Stockett was born and raised in Jackso...,https://images.gr-assets.com/authors/123345810...,Historical Fiction,http://www.kathrynstockett.com,None,4.47,"2,666,191","89,477",9,[],September 2008,NaN
9,F. Scott Fitzgerald,"September 24, 1896","December 21, 1940","in St. Paul, Minnesota, The United States",Francis Scott Key Fitzgerald was an American w...,https://images.gr-assets.com/authors/151786400...,"Fiction, Short Stories",None,"John Keats, Sherwood Anderson, Edith Wharton, ...",3.91,"5,464,678","130,942","2,228",[Специално издание Biograph],NaN,NaN


In [8]:
authors_df.to_csv('authors.csv', index = True)